<a href="https://colab.research.google.com/github/shuta13/sklearn-sandbox-colab/blob/main/scikit_learn_sandbox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ref: https://colab.research.google.com/github/chainer/tutorials/blob/master/ja/09_Introduction_to_Scikit-learn.ipynb

In [ ]:
!pip install -U scikit-learn

In [ ]:
from sklearn.datasets import load_boston

dataset = load_boston()

In [ ]:
x = dataset.data
t = dataset.target

In [ ]:
x.shape

(506, 13)

In [ ]:
t.shape

(506,)

In [ ]:
from sklearn.model_selection import train_test_split

# 全サンプルからランダムに70%を訓練データとして抽出し、残った30%のデータをテストデータとして返す
# random_state に毎回同じ同じ整数を与えると実行のたびに結果が変わることがなくなる
x_train, x_test, t_train, t_test = train_test_split(x, t, test_size=0.3, random_state=0)

In [ ]:
from sklearn.linear_model import LinearRegression

reg_model = LinearRegression()

In [ ]:
# モデルの訓練
reg_model.fit(x_train, t_train)

LinearRegression()

In [ ]:
# 訓練後のパラメータ w(重み)
reg_model.coef_

array([-1.21310401e-01,  4.44664254e-02,  1.13416945e-02,  2.51124642e+00,
       -1.62312529e+01,  3.85906801e+00, -9.98516565e-03, -1.50026956e+00,
        2.42143466e-01, -1.10716124e-02, -1.01775264e+00,  6.81446545e-03,
       -4.86738066e-01])

In [ ]:
# 訓練後のバイアス b
reg_model.intercept_

37.93710774183309

In [ ]:
# 訓練済みモデルを用いて訓練用データセットで計算した決定係数を求める
# 入力値と目標値を与える
reg_model.score(x_train, t_train)

0.7645451026942549

やる気がなくなったので重回帰分析終わり

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html
# ↑をボストンのデータで学習

from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

regr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
regr.fit(x_train, t_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svr', SVR(epsilon=0.2))])

In [ ]:
regr.predict(x_test)

array([25.14026237, 19.02907465, 24.36767296, 13.53374266, 20.95883182,
       19.41489106, 20.89982247, 20.07000678, 19.00683567, 18.85965491,
       20.788081  , 13.65105369, 14.87679672, 13.3374993 , 26.88157904,
       29.88842135, 21.49027712, 31.34915606, 28.83895926, 21.48232931,
       23.80849272, 21.81791184, 18.8352386 , 27.96899922, 20.54142766,
       17.87385466, 16.96447577, 18.54391008, 33.47151933, 18.62828988,
       15.40195545, 17.39201543, 19.37755169, 21.20803717, 25.82025237,
       21.07323925, 12.33119513, 22.36915916, 14.07973701, 13.47269493,
       23.67886537, 20.19279478, 22.90706571, 15.08596328, 24.95158739,
       23.17248401, 18.97719187, 21.44966319, 16.94881921, 22.93938085,
       20.92292256, 19.68449947, 21.39528485, 29.20644376, 15.31191596,
       21.15633401, 20.0429304 , 18.20681521, 19.58745358, 21.47171818,
       20.72261573, 20.90562968, 31.54468769, 28.31041928, 18.47856558,
       29.91706833, 16.24239145, 19.97264209, 14.11365795, 22.06